# Setup

In [1]:
!git clone https://github.com/jmerullo/lm_vector_arithmetic.git

Cloning into 'lm_vector_arithmetic'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 35 (delta 10), reused 21 (delta 4), pack-reused 0
Receiving objects: 100% (35/35), 236.07 KiB | 19.67 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
cd lm_vector_arithmetic

/content/lm_vector_arithmetic


In [3]:
%%capture
!pip install -r requirements.txt

In [4]:
from modeling import *
import matplotlib.pyplot as plt

In [5]:
model, tokenizer = load_gpt2('gpt2')
model = model.float()
wrapper = GPT2Wrapper(model, tokenizer)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

# Functions

In [6]:
def tokenize(text):
    inp_ids = wrapper.tokenize(text)
    str_toks = wrapper.list_decode(inp_ids[0])
    return inp_ids, str_toks

In [7]:
def get_tokIDS(test_text):
    test_ids, test_toks = tokenize(test_text)
    logits = wrapper.get_layers(test_ids)
    wrapper.print_top(logits[1:]) #skip the first embedding layer b/c it just reverses to input
    return test_ids

In [8]:
def get_logits(test_text):
    test_ids, test_toks = tokenize(test_text)
    logits = wrapper.get_layers(test_ids)
    # wrapper.print_top(logits[1:]) #skip the first embedding layer b/c it just reverses to input
    return logits

In [9]:
def create_layer_list(X, Y):
    layer_list = []
    for layer in range(X, Y+1):
        layer_list.append("Layer {}".format(layer))
    return layer_list

from tabulate import tabulate
def create_table(list1, list2):
    table_data = zip(list1, list2)
    headers = ["Layer", "Orig Top Token"]
    table = tabulate(table_data, headers, tablefmt="grid")
    return table

import torch.nn.functional as F
def get_decoded(logits, k=10):
    output_list = []
    for i,layer in enumerate(logits):
        output_list.append( wrapper.tokenizer.decode(F.softmax(layer,dim=-1).argsort(descending=True)[:k]) )
    return output_list

def format_to_table(tok_ids):
    out = wrapper.model(input_ids = tok_ids, output_hidden_states=True)
    logits = out.logits
    hidden_states = out.hidden_states
    hidden_states = list(hidden_states)[1:]

    orig_layer_logits = wrapper.layer_decode(hidden_states)
    orig_layer_logits = torch.stack(orig_layer_logits).squeeze(-1)

    orig_decoded = get_decoded(orig_layer_logits)
    table_output = create_table(create_layer_list(0, 23), orig_decoded)
    print(table_output)

# umbed fns

In [ ]:
def test_prompt(
    prompt: str,
    answer: str,
    model,  # Can't give type hint due to circular imports
    prepend_space_to_answer: Optional[bool] = True,
    print_details: Optional[bool] = True,
    prepend_bos: Optional[bool] = False, # key change
    top_k: Optional[int] = 10,
) -> None:
    """
    Args:
        prompt:
            The prompt string, e.g. "Why did the elephant cross the".
        answer:
            The answer, e.g. "road". Note that if you set prepend_space_to_answer to False, you need
            to think about if you have a space before the answer here (as e.g. in this example the
            answer may really be " road" if the prompt ends without a trailing space).
        model:
            The model.
        prepend_space_to_answer:
            Whether or not to prepend a space to the answer. Note this will only ever prepend a
            space if the answer doesn't already start with one.
        print_details:
            Print the prompt (as a string but broken up by token), answer and top k tokens (all
            with logit, rank and probability).
        prepend_bos:
            Overrides self.cfg.default_prepend_bos if set. Whether to prepend
            the BOS token to the input (applicable when input is a string). Models generally learn
            to use the BOS token as a resting place for attention heads (i.e. a way for them to be
            "turned off"). This therefore often improves performance slightly.
        top_k:
            Top k tokens to print details of (when print_details is set to True).

    Returns:
        None (just prints the results directly).
    """
    if prepend_space_to_answer and not answer.startswith(" "):
        answer = " " + answer
    # GPT-2 often treats the first token weirdly, so lets give it a resting position
    prompt_tokens = model.to_tokens(prompt, prepend_bos=prepend_bos)
    answer_tokens = model.to_tokens(answer, prepend_bos=False)
    tokens = torch.cat((prompt_tokens, answer_tokens), dim=1)
    prompt_str_tokens = model.to_str_tokens(prompt, prepend_bos=prepend_bos)
    answer_str_tokens = model.to_str_tokens(answer, prepend_bos=False)
    prompt_length = len(prompt_str_tokens)
    answer_length = len(answer_str_tokens)
    if print_details:
        print("Tokenized prompt:", prompt_str_tokens)
        print("Tokenized answer:", answer_str_tokens)
    # logits = remove_batch_dim(model(tokens))  # key change
    logits = remove_batch_dim(model(prompt_tokens))  # key change
    probs = logits.softmax(dim=-1)

    # key changes
    logits = logits[0]
    probs = probs[0]

    answer_ranks = []
    for index in range(prompt_length, prompt_length + answer_length):
        answer_token = tokens[0, index]
        answer_str_token = answer_str_tokens[index - prompt_length]
        # Offset by 1 because models predict the NEXT token
        token_probs = probs[index - 1]
        sorted_token_probs, sorted_token_values = token_probs.sort(descending=True)
        # Janky way to get the index of the token in the sorted list - I couldn't find a better way?
        correct_rank = torch.arange(len(sorted_token_values))[
            (sorted_token_values == answer_token).cpu()
        ].item()
        answer_ranks.append((answer_str_token, correct_rank))
        if print_details:
            # String formatting syntax - the first number gives the number of characters to pad to, the second number gives the number of decimal places.
            # rprint gives rich text printing
            print(
                f"Performance on answer token:\n[b]Rank: {correct_rank: <8} Logit: {logits[index-1, answer_token].item():5.2f} Prob: {token_probs[answer_token].item():6.2%} Token: |{answer_str_token}|[/b]"
            )
            for i in range(top_k):
                print(
                    f"Top {i}th token. Logit: {logits[index-1, sorted_token_values[i]].item():5.2f} Prob: {sorted_token_probs[i].item():6.2%} Token: |{model.to_string(sorted_token_values[i])}|"
                )
    print(f"[b]Ranks of the answer tokens:[/b] {answer_ranks}")

# Test Template

In [18]:
test_text = """1 2 3 4"""
layer_logits = get_logits(test_text)

0 thteenthx 4 3teen3454 239
1 thteenth54x3934teen384174
2 thx3454GHz388639teen74
3 3454thGHz86ts743979x
4 tsms34thths54 -iel86 4
5 ts 4 - 3 5ths 6 +ms34
6  4★ 3 5 6 2 Tycoon >>> +ts
7  4 5 6 3 0 2 Tycoon 1ths 8
8  4 5 3 6 1 2 8 7★ 9
9  5 4 6 3 75 1 9 0★
10  5 6 4 3 7 1 05 9/
11  5 4 1 6 3 0/ 7
5


In [24]:
layer_logits

tensor([[ 43.8442,  48.0193,  52.5031,  ...,  47.6105,  41.7179,  43.1619],
        [ -3.5105,   0.4111,  -2.1912,  ..., -13.0544,  -8.5994,  -1.3726],
        [ -5.0367,  -1.0030,  -4.3191,  ..., -17.0995, -12.0476,  -3.1856],
        ...,
        [-24.6106, -23.8827, -23.6106,  ..., -37.4969, -37.8311, -22.5237],
        [-31.6502, -30.8153, -31.5918,  ..., -41.6925, -40.8858, -28.3813],
        [-40.0663, -39.1234, -40.0680,  ..., -47.4770, -46.6980, -38.0356]],
       grad_fn=<SqueezeBackward1>)

In [28]:
wrapper.tokenizer.decode(F.softmax(layer_logits[2],dim=-1).argsort(descending=True)[:2])

'thteenth'

In [29]:
wrapper.tokenizer.decode(F.softmax(layer_logits[2],dim=-1).argsort(descending=True)[:1])

'th'

In [30]:
top_5_at_layer = []
for i in range(5):
    out = wrapper.tokenizer.decode(F.softmax(layer_logits[2],dim=-1).argsort(descending=True)[i])
    top_5_at_layer.append(out)
top_5_at_layer

['th', 'teenth', '54', 'x', '39']

In [10]:
def get_decoded_indiv_toks(logits, k=10):
    output_list = []
    for i, layer in enumerate(logits):
        top_5_at_layer = []
        for j in range(5):
            # print(i, j)
            out = wrapper.tokenizer.decode(F.softmax(logits[i],dim=-1).argsort(descending=True)[j])
            top_5_at_layer.append(out)
        output_list.append( top_5_at_layer )
    return output_list

In [21]:
tok_logit_lens = get_decoded(layer_logits)
tok_logit_lens

[' 4 livest mathemat myster horizont challengtheless destro 3 5',
 'thteenthx 4 3teen3454 239',
 'thteenth54x3934teen384174',
 'thx3454GHz388639teen74',
 '3454thGHz86ts743979x',
 'tsms34thths54 -iel86 4',
 'ts 4 - 3 5ths 6 +ms34',
 ' 4★ 3 5 6 2 Tycoon >>> +ts',
 ' 4 5 6 3 0 2 Tycoon 1ths 8',
 ' 4 5 3 6 1 2 8 7★ 9',
 ' 5 4 6 3 75 1 9 0★',
 ' 5 6 4 3 7 1 05 9/',
 ' 5 4 1 6 3 0/ 7\n5']

In [43]:
tok_logit_lens = get_decoded_indiv_toks(layer_logits)
for i, tokouts in enumerate(tok_logit_lens):
    print(i, tokouts)

0 [' 4', ' livest', ' mathemat', ' myster', ' horizont']
1 ['th', 'teenth', 'x', ' 4', ' 3']
2 ['th', 'teenth', '54', 'x', '39']
3 ['th', 'x', '34', '54', 'GHz']
4 ['34', '54', 'th', 'GHz', '86']
5 ['ts', 'ms', '34', 'th', 'ths']
6 ['ts', ' 4', ' -', ' 3', ' 5']
7 [' 4', '★', ' 3', ' 5', ' 6']
8 [' 4', ' 5', ' 6', ' 3', ' 0']
9 [' 4', ' 5', ' 3', ' 6', ' 1']
10 [' 5', ' 4', ' 6', ' 3', ' 7']
11 [' 5', ' 6', ' 4', ' 3', ' 7']
12 [' 5', ' 4', ' 1', ' 6', ' 3']


In [54]:
test_ids = get_tokIDS(test_text)
format_to_table(test_ids)

0  2015 May 2014 April June Aug 2013 July 2017 2016
1  2015 2014 2017 2013 2016 29 2018 27 May 24
2  2015 2014 29 2017 28 27 30 2013 2018 2016
3  29 2015 27 28 30 24 2014 25 26 23
4  24 29 2014 2015 2018 2017 27 30 28 26
5  29 24 30 28 27 4 3 26 2014 6
6  29 24 30 2014 28 26 2018 31 27 2015
7  2014 29 April 24 2015 2018 2019 2013 4 28
8  April 2014 May March 2015 June September 2017 October Apr
9  May April June 2015 5 July 2005 September March November
10  May June July AprilMay November September 5 March October
11  May June July 5
 April - MarchMay November
+----------+-----------------------------------------------------------+
| Layer    | Orig Top Token                                            |
+==========+===========================================================+
| Layer 0  | 2015 May 2014 April June Aug 2013 July 2017 2016          |
+----------+-----------------------------------------------------------+
| Layer 1  | 2015 2014 2017 2013 2016 29 2018 27 May 24             

# loop and get freqs

In [46]:
test_text = "1 2 3 4"
layer_logits = get_logits(test_text)
tok_logit_lens = get_decoded_indiv_toks(layer_logits)
for i, tokouts in enumerate(tok_logit_lens):
    print(i, tokouts)

0 [' 4', ' livest', ' mathemat', ' myster', ' horizont']
1 ['th', 'teenth', 'x', ' 4', ' 3']
2 ['th', 'teenth', '54', 'x', '39']
3 ['th', 'x', '34', '54', 'GHz']
4 ['34', '54', 'th', 'GHz', '86']
5 ['ts', 'ms', '34', 'th', 'ths']
6 ['ts', ' 4', ' -', ' 3', ' 5']
7 [' 4', '★', ' 3', ' 5', ' 6']
8 [' 4', ' 5', ' 6', ' 3', ' 0']
9 [' 4', ' 5', ' 3', ' 6', ' 1']
10 [' 5', ' 4', ' 6', ' 3', ' 7']
11 [' 5', ' 6', ' 4', ' 3', ' 7']
12 [' 5', ' 4', ' 1', ' 6', ' 3']


In [50]:
prompts = ["1 2 3 4", "one two three four", "January February March April"]
for test_text in prompts:
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' 4', ' livest', ' mathemat', ' myster', ' horizont']
0 ['th', 'teenth', 'x', ' 4', ' 3']
1 ['th', 'teenth', '54', 'x', '39']
2 ['th', 'x', '34', '54', 'GHz']
3 ['34', '54', 'th', 'GHz', '86']
4 ['ts', 'ms', '34', 'th', 'ths']
5 ['ts', ' 4', ' -', ' 3', ' 5']
6 [' 4', '★', ' 3', ' 5', ' 6']
7 [' 4', ' 5', ' 6', ' 3', ' 0']
8 [' 4', ' 5', ' 3', ' 6', ' 1']
9 [' 5', ' 4', ' 6', ' 3', ' 7']
10 [' 5', ' 6', ' 4', ' 3', ' 7']
11 [' 5', ' 4', ' 1', ' 6', ' 3']


-1 [' four', ' livest', ' three', ' five', ' six']
0 ['teenth', 'teen', ' hundred', 'een', 'some']
1 ['teen', 'teenth', ' hundred', 'some', ' years']
2 ['teen', 'teenth', ' hundred', 'some', ' months']
3 ['teen', 'teenth', ' hundred', 'some', ' times']
4 ['teen', 'teenth', ' hundred', ' months', ' times']
5 ['teen', 'teenth', 'ths', ' inches', ' hundred']
6 ['teen', 'teenth', 'ths', ' hundred', '-']
7 ['teen', 'ths', 'teenth', ' hundred', '-|']
8 ['teen', 'ths', ' 4', 'teenth', ' four']
9 [' five', ' Five', ' 5', ' seven', ' six'

# pure seq prompts

In [55]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'corr': str(i+4),
            'incorr': str(i+3),
            'text': f"{i} {i+1} {i+2} {i+3}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(1, 9)

In [56]:
len(prompts_list)

8

In [57]:
prompts_list

[{'corr': '5', 'incorr': '4', 'text': '1 2 3 4'},
 {'corr': '6', 'incorr': '5', 'text': '2 3 4 5'},
 {'corr': '7', 'incorr': '6', 'text': '3 4 5 6'},
 {'corr': '8', 'incorr': '7', 'text': '4 5 6 7'},
 {'corr': '9', 'incorr': '8', 'text': '5 6 7 8'},
 {'corr': '10', 'incorr': '9', 'text': '6 7 8 9'},
 {'corr': '11', 'incorr': '10', 'text': '7 8 9 10'},
 {'corr': '12', 'incorr': '11', 'text': '8 9 10 11'}]

In [58]:
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' 4', ' livest', ' mathemat', ' myster', ' horizont']
0 ['th', 'teenth', 'x', ' 4', ' 3']
1 ['th', 'teenth', '54', 'x', '39']
2 ['th', 'x', '34', '54', 'GHz']
3 ['34', '54', 'th', 'GHz', '86']
4 ['ts', 'ms', '34', 'th', 'ths']
5 ['ts', ' 4', ' -', ' 3', ' 5']
6 [' 4', '★', ' 3', ' 5', ' 6']
7 [' 4', ' 5', ' 6', ' 3', ' 0']
8 [' 4', ' 5', ' 3', ' 6', ' 1']
9 [' 5', ' 4', ' 6', ' 3', ' 7']
10 [' 5', ' 6', ' 4', ' 3', ' 7']
11 [' 5', ' 4', ' 1', ' 6', ' 3']


-1 [' livest', ' 5', ' destro', 'theless', ' mathemat']
0 ['th', ' 5', ' 3', 'x', ' times']
1 ['th', 'x', '54', '45', '43']
2 ['th', '34', 'x', 'min', '54']
3 ['34', 'th', '90', '54', '85']
4 ['ths', 'ts', ' -', 'min', '34']
5 [' -', 'ts', ' 3', 'ths', ' 5']
6 [' 5', ' 4', ' 3', ' 6', '★']
7 [' 5', ' 6', ' 4', ' 3', ' 9']
8 [' 5', ' 6', ' 4', ' 3', ' 7']
9 [' 6', ' 5', ' 4', ' 7', '6']
10 [' 6', ' 5', ' 4', ' 7', ' 8']
11 [' 6', ' 5', ' 4', ' 7', ' 0']


-1 [' 6', ' livest', ' destro', ' mathemat', ' challeng']
0 ['th', '36', '31

# pure seq prompts nw

In [24]:
def generate_prompts_list(x ,y):
    words = [' one', ' two', ' three', ' four', ' five', ' six', ' seven', ' eight', ' nine', ' ten', ' eleven', ' twelve', ' thirteen', ' fourteen', ' fifteen', ' sixteen', ' seventeen', ' eighteen', ' nineteen', ' twenty']
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': words[i],
            'S2': words[i+1],
            'S3': words[i+2],
            'S4': words[i+3],
            'corr': words[i+4],
            'incorr': words[i+3],  # this is arbitrary
            'text': f"{words[i]}{words[i+1]}{words[i+2]}{words[i+3]}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(0, 8)

In [25]:
len(prompts_list)

8

In [26]:
prompts_list

[{'S1': ' one',
  'S2': ' two',
  'S3': ' three',
  'S4': ' four',
  'corr': ' five',
  'incorr': ' four',
  'text': ' one two three four'},
 {'S1': ' two',
  'S2': ' three',
  'S3': ' four',
  'S4': ' five',
  'corr': ' six',
  'incorr': ' five',
  'text': ' two three four five'},
 {'S1': ' three',
  'S2': ' four',
  'S3': ' five',
  'S4': ' six',
  'corr': ' seven',
  'incorr': ' six',
  'text': ' three four five six'},
 {'S1': ' four',
  'S2': ' five',
  'S3': ' six',
  'S4': ' seven',
  'corr': ' eight',
  'incorr': ' seven',
  'text': ' four five six seven'},
 {'S1': ' five',
  'S2': ' six',
  'S3': ' seven',
  'S4': ' eight',
  'corr': ' nine',
  'incorr': ' eight',
  'text': ' five six seven eight'},
 {'S1': ' six',
  'S2': ' seven',
  'S3': ' eight',
  'S4': ' nine',
  'corr': ' ten',
  'incorr': ' nine',
  'text': ' six seven eight nine'},
 {'S1': ' seven',
  'S2': ' eight',
  'S3': ' nine',
  'S4': ' ten',
  'corr': ' eleven',
  'incorr': ' ten',
  'text': ' seven eight nine 

In [27]:
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' four', ' livest', ' three', ' five', ' six']
0 ['teenth', 'teen', ' hundred', 'een', 'some']
1 ['teenth', 'teen', ' hundred', 'some', 'een']
2 ['teen', 'teenth', ' hundred', 'some', ' million']
3 ['teen', 'teenth', ' hundred', 'some', ' times']
4 ['teen', 'teenth', ' hundred', ' months', ' times']
5 ['teen', 'teenth', 'ths', ' hundred', ' inches']
6 ['teen', 'teenth', 'ths', ' hundred', '-']
7 ['teen', 'teenth', ' hundred', 'ths', '-|']
8 ['teen', 'ths', 'teenth', ' 4', ' four']
9 [' five', ' 5', ' seven', ' Five', ' six']
10 [' five', ' six', ' Five', ' seven', ' four']
11 [' five', ' six', ' Five', ' seven', ' four']


-1 [' five', ' livest', 'five', ' seven', ' four']
0 ['teen', 'teenth', ' hundred', ' years', ' months']
1 ['teen', 'teenth', ' hundred', ' minutes', ' months']
2 ['teen', ' hundred', 'teenth', ' minutes', ' million']
3 ['teen', ' hundred', 'teenth', ' minutes', ' times']
4 ['teen', ' hundred', ' minutes', 'teenth', ' years']
5 ['ths', ' hundred', 'teen', ' thous

# pure seq prompts months

In [30]:
def generate_prompts_list(x ,y):
    words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': words[i],
            'S2': words[i+1],
            'S3': words[i+2],
            'S4': words[i+3],
            'corr': words[i+4],
            'incorr': words[i+3],  # this is arbitrary
            'text': f"{words[i]}{words[i+1]}{words[i+2]}{words[i+3]}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(0, 8)

In [31]:
len(prompts_list)

8

In [32]:
prompts_list

[{'S1': 'January',
  'S2': 'February',
  'S3': 'March',
  'S4': 'April',
  'corr': 'May',
  'incorr': 'April',
  'text': 'JanuaryFebruaryMarchApril'},
 {'S1': 'February',
  'S2': 'March',
  'S3': 'April',
  'S4': 'May',
  'corr': 'June',
  'incorr': 'May',
  'text': 'FebruaryMarchAprilMay'},
 {'S1': 'March',
  'S2': 'April',
  'S3': 'May',
  'S4': 'June',
  'corr': 'July',
  'incorr': 'June',
  'text': 'MarchAprilMayJune'},
 {'S1': 'April',
  'S2': 'May',
  'S3': 'June',
  'S4': 'July',
  'corr': 'August',
  'incorr': 'July',
  'text': 'AprilMayJuneJuly'},
 {'S1': 'May',
  'S2': 'June',
  'S3': 'July',
  'S4': 'August',
  'corr': 'September',
  'incorr': 'August',
  'text': 'MayJuneJulyAugust'},
 {'S1': 'June',
  'S2': 'July',
  'S3': 'August',
  'S4': 'September',
  'corr': 'October',
  'incorr': 'September',
  'text': 'JuneJulyAugustSeptember'},
 {'S1': 'July',
  'S2': 'August',
  'S3': 'September',
  'S4': 'October',
  'corr': 'November',
  'incorr': 'October',
  'text': 'JulyAugust

In [33]:
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 ['April', 'June', 'February', 'October', 'September']
0 [' 2015', ' 2014', ' May', ' Aug', ' 2013']
1 [' 2015', ' 2014', ' 2017', ' 2013', ' 2016']
2 [' 2014', ' 2015', ' 2013', ' 29', ' 2017']
3 [' 2015', ' 2014', ' 29', ' 2017', ' 28']
4 [' 2014', ' 2015', ' 29', ' 2017', ' 30']
5 [' 29', ' 30', ' 2014', ' 28', '/']
6 [' 2014', ' 29', '/', ' 30', ' 2015']
7 ['April', 'December', 'June', 'September', 'October']
8 ['April', 'June', 'March', 'September', 'October']
9 ['May', 'April', 'June', 'Apr', 'October']
10 ['May', 'June', 'April', 'July', 'October']
11 ['May', 'June', 'July', 'April', 'October']


-1 ['May', ' livest', 'June', 'April', 'Apr']
0 [' May', 'May', 'flower', ' 2015', 'Aug']
1 [' 2015', ' 2014', ' May', ' 2013', 'flower']
2 [' 2014', ' 2015', 'flower', ' 2011', ' 2013']
3 [' 2015', 'hem', ' 2014', 'nard', ' 31']
4 ['weather', 'hem', ' 2014', 'yy', ' 2015']
5 ['weather', '/', ']', ' 29', 'way']
6 ['flower', 'May', 'June', 'December', 'hem']
7 ['hem', 'flower', 'May', 

# nonpure digits prompts

In [60]:
import pickle

task = "digits"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:10] #768 512

len(prompts_list)

Van done in 1. Hat done in 2. Ring done in 3. Desk done in 4. Sun done in
Oil lost in 1. Apple lost in 2. Tree lost in 3. Snow lost in 4. Apple lost in
Marcus born in 1. Victoria born in 2. George born in 3. Brandon born in 4. Jamie born in


30

In [61]:
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' a', ' my']
2 [' order', ' the', ' conjunction', ' front', ' vain']
3 [' order', ' the', ' vain', ' conjunction', ' accordance']
4 [' order', ' the', ' conjunction', ' vain', ' front']
5 [' order', ' 3', ' conjunction', ' 2', ' 18']
6 [' 3', ' 1', ' 2', ' 5', ' 4']
7 [' 3', ' 4', ' 1', ' 2', ' 5']
8 [' 4', ' 5', ' 3', ' 1', ' 2']
9 [' 5', ' 6', ' 4', ' 1', ' 7']
10 [' 5', ' 6', ' 1', ' 7', ' 4']
11 [' 5', ' 1', ' 6', ' 4', ' 7']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' conjunction', ' a']
1 [' order', ' the', ' conjunction', ' my', ' particular']
2 [' order', ' the', ' conjunction', ' accordance', ' a']
3 [' order', ' conjunction', ' the', ' accordance', ' advance']
4 [' order', ' conjunction', ' accordance', ' the', ' lieu']
5 [' order', ' conjunction', ' lieu', ' acc

In [74]:
num_corr = 0
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    if int(tok_logit_lens[9][0].replace(' ', '')) < int(tok_logit_lens[10][0].replace(' ', '')):
        if tok_logit_lens[10][0] == pd['corr']:
            num_corr += 1
    # for i, tokouts in enumerate(tok_logit_lens):
    #     print(i-1, tokouts)
    # print('\n')
num_corr

30

In [75]:
prompts_list[0]['text']

'Van done in 1. Hat done in 2. Ring done in 3. Desk done in 4. Sun done in'

In [72]:
pd['text']

'Anne born in 2. Chelsea born in 3. Jeremy born in 4. Craig born in 5. Elizabeth born in'

In [71]:
tok_logit_lens[9][0] == pd['corr']

False

In [67]:
tok_logit_lens[9][0]

' 5'

In [73]:
tok_logit_lens[10][0]

' 6'

In [68]:
pd['corr']

' 6'

In [64]:
tok_logit_lens[8]

[' 3', ' 5', ' 2', ' 4', ' 1']

In [63]:
tok_logit_lens[9]

[' 5', ' 6', ' 4', ' 7', ' 3']

# nw

In [76]:
import pickle

task = "nw"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:10] #768 512

len(prompts_list)

Van done in one. Hat done in two. Ring done in three. Desk done in four. Sun done in
Oil lost in one. Apple lost in two. Tree lost in three. Snow lost in four. Apple lost in
Marcus born in one. Victoria born in two. George born in three. Brandon born in four. Jamie born in


30

In [84]:
num_words = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
                 "eleven", "twelve"]

anomolies = []
num_corr = 0
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    a = num_words.index(tok_logit_lens[9][0].replace(' ', ''))
    b= num_words.index(tok_logit_lens[10][0].replace(' ', ''))
    if int(a) < int(b):
        if tok_logit_lens[10][0] == pd['corr']:
            num_corr += 1
        else:
            anomolies.append(pd)
    else:
            anomolies.append(pd)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' front']
1 [' order', ' the', ' conjunction', ' my', ' a']
2 [' order', ' the', ' front', ' vain', ' conjunction']
3 [' order', ' the', ' vain', ' conjunction', ' accordance']
4 [' order', ' the', ' conjunction', ' vain', ' relation']
5 [' order', ' conjunction', ' lieu', ' vain', ' the']
6 [' conjunction', ' order', ' lieu', ' the', ' a']
7 [' order', ' three', ' another', ' vain', ' third']
8 [' four', ' three', ' five', ' six', ' seven']
9 [' five', ' six', ' seven', ' four', ' fifth']
10 [' five', ' six', ' seven', ' Five', ' one']
11 [' five', ' six', ' one', ' seven', ' four']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' conjunction', ' a']
1 [' order', ' the', ' conjunction', ' my', ' particular']
2 [' order', ' the', ' conjunction', ' accordance', ' front']
3 [' order', ' the', ' conjunction', ' accordance', ' vain']

In [85]:
num_corr

28

In [86]:
anomolies

[{'S1': 'three',
  'S2': 'four',
  'S3': 'five',
  'S4': 'six',
  'corr': ' seven',
  'incorr': ' six',
  'text': 'Ring done in three. Moon done in four. Queen done in five. Book done in six. Rose done in'},
 {'S1': 'three',
  'S2': 'four',
  'S3': 'five',
  'S4': 'six',
  'corr': ' seven',
  'incorr': ' six',
  'text': 'Wallet lost in three. Jam lost in four. Wall lost in five. Watch lost in six. Train lost in'}]

In [87]:
for pd in anomolies:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' vain', ' particular']
2 [' order', ' the', ' vain', ' front', ' conjunction']
3 [' order', ' the', ' vain', ' conjunction', ' accordance']
4 [' order', ' the', ' relation', ' vain', ' conjunction']
5 [' order', ' vain', ' lieu', ' conjunction', ' the']
6 [' order', ' three', ' conjunction', ' lieu', ' a']
7 [' three', ' five', ' seven', ' nine', ' order']
8 [' seven', ' six', ' five', ' four', ' eight']
9 [' seven', ' eight', ' six', ' nine', ' seventh']
10 [' seven', ' eight', ' nine', ' six', ' nineteen']
11 [' seven', ' eight', ' six', ' nine', ' ten']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' the', ' order', ' a', ' particular', ' front']
1 [' order', ' the', ' a', ' front', ' my']
2 [' order', ' the', ' a', ' vain', ' front']
3 [' order', ' the', ' vain', ' accordance', ' a']
4 [' order', ' vain'

# nw 1536- 10 ten anomoly

In [11]:
import pickle

task = "nw"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:512] #768 512

len(prompts_list)

Van done in one. Hat done in two. Ring done in three. Desk done in four. Sun done in
Oil lost in one. Apple lost in two. Tree lost in three. Snow lost in four. Apple lost in
Marcus born in one. Victoria born in two. George born in three. Brandon born in four. Jamie born in


1536

In [12]:
num_words = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
                 "eleven", "twelve"]

anomolies = []
num_corr = 0
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    a = num_words.index(tok_logit_lens[9][0].replace(' ', ''))
    b= num_words.index(tok_logit_lens[10][0].replace(' ', ''))
    if int(a) < int(b):
        if tok_logit_lens[10][0] == pd['corr']:
            num_corr += 1
        else:
            anomolies.append(pd)
    else:
            anomolies.append(pd)
    # for i, tokouts in enumerate(tok_logit_lens):
    #     print(i-1, tokouts)
    # print('\n')

ValueError: ignored

In [14]:
pd

{'S1': 'six',
 'S2': 'seven',
 'S3': 'eight',
 'S4': 'nine',
 'corr': ' ten',
 'incorr': ' nine',
 'text': 'Tree done in six. Ball done in seven. Wallet done in eight. Tree done in nine. Apple done in'}

In [13]:
tok_logit_lens

[[' challeng', ' mathemat', ' arrang', ' corrid', ' destro'],
 [' order', ' the', ' particular', ' a', ' front'],
 [' order', ' the', ' conjunction', ' particular', ' vain'],
 [' order', ' the', ' front', ' conjunction', ' vain'],
 [' order', ' the', ' conjunction', ' vain', ' accordance'],
 [' order', ' vain', ' conjunction', ' the', ' front'],
 [' order', ' conjunction', ' vain', ' lieu', ' the'],
 [' order', ' conjunction', ' vain', ' lieu', ' 15'],
 [' order', ' 12', ' 17', ' vain', ' 13'],
 [' nine', ' eight', ' seven', ' 10', ' ten'],
 [' 10', ' ten', ' tenth', ' nine', ' twelve'],
 [' ten', ' 10', ' twelve', ' eleven', ' fifteen'],
 [' ten', ' 10', ' a', ' fifteen', ' twenty']]

In [ ]:
num_corr

In [ ]:
# anomolies

In [ ]:
# for pd in anomolies:
#     test_text = pd['text']
#     layer_logits = get_logits(test_text)
#     tok_logit_lens = get_decoded_indiv_toks(layer_logits)
#     for i, tokouts in enumerate(tok_logit_lens):
#         print(i-1, tokouts)
#     print('\n')

# nw 1536

In [ ]:
import pickle

task = "nw"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:512] #768 512

len(prompts_list)

Van done in one. Hat done in two. Ring done in three. Desk done in four. Sun done in
Oil lost in one. Apple lost in two. Tree lost in three. Snow lost in four. Apple lost in
Marcus born in one. Victoria born in two. George born in three. Brandon born in four. Jamie born in


1536

In [15]:
num_words = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
                 "eleven", "twelve"]

anomolies = []
num_corr = 0
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    try:
        a = num_words.index(tok_logit_lens[9][0].replace(' ', ''))
        b= num_words.index(tok_logit_lens[10][0].replace(' ', ''))
        if int(a) < int(b):
            if tok_logit_lens[10][0] == pd['corr']:
                num_corr += 1
            else:
                anomolies.append(pd)
        else:
                anomolies.append(pd)
    except:
        anomolies.append(pd)
    # for i, tokouts in enumerate(tok_logit_lens):
    #     print(i-1, tokouts)
    # print('\n')

In [16]:
num_corr

1396

In [23]:
len(prompts_list)

1536

In [28]:
anomolies[0]

{'S1': 'three',
 'S2': 'four',
 'S3': 'five',
 'S4': 'six',
 'corr': ' seven',
 'incorr': ' six',
 'text': 'Ring done in three. Moon done in four. Queen done in five. Book done in six. Rose done in'}

In [29]:
anomolies[-1]

{'S1': 'six',
 'S2': 'seven',
 'S3': 'eight',
 'S4': 'nine',
 'corr': ' ten',
 'incorr': ' nine',
 'text': 'Marcus born in six. April born in seven. Ryan born in eight. Jeremy born in nine. Jim born in'}

In [19]:
for pd in anomolies[:2]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' vain', ' particular']
2 [' order', ' the', ' vain', ' front', ' conjunction']
3 [' order', ' the', ' vain', ' conjunction', ' accordance']
4 [' order', ' the', ' relation', ' vain', ' conjunction']
5 [' order', ' vain', ' lieu', ' conjunction', ' the']
6 [' order', ' three', ' conjunction', ' lieu', ' a']
7 [' three', ' five', ' seven', ' nine', ' order']
8 [' seven', ' six', ' five', ' four', ' eight']
9 [' seven', ' eight', ' six', ' nine', ' seventh']
10 [' seven', ' eight', ' nine', ' six', ' nineteen']
11 [' seven', ' eight', ' six', ' nine', ' ten']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' front']
1 [' order', ' the', ' conjunction', ' vain', ' particular']
2 [' order', ' the', ' vain', ' conjunction', ' front']
3 [' order', ' vain', ' the', ' accordance',

In [21]:
for pd in anomolies[-2:]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' front']
1 [' the', ' order', ' a', ' conjunction', ' particular']
2 [' the', ' order', ' a', ' conjunction', ' front']
3 [' the', ' order', ' accordance', ' conjunction', ' a']
4 [' the', ' front', ' order', ' conjunction', ' his']
5 [' order', ' 18', ' 17', ' front', ' conjunction']
6 [' front', ' 10', ' order', ' 3', ' three']
7 [' nine', ' seven', ' 9', ' eight', ' 12']
8 [' nine', ' eight', ' 10', ' ten', ' seven']
9 [' 10', ' ten', ' nine', ' twelve', ' tenth']
10 [' ten', ' twelve', ' eleven', ' 10', ' thirteen']
11 [' ten', ' 10', ' twelve', ' eleven', ' fifteen']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' the', ' order', ' particular', ' conjunction', ' a']
2 [' the', ' order', ' a', ' conjunction', ' front']
3 [' the', ' order', ' accordance', ' his', ' conjunction']
4 [' the', ' order', 

# months 16

In [58]:
import pickle

task = "months"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:512] #768 512

len(prompts_list)

Van done in January. Hat done in February. Ring done in March. Desk done in April. Sun done in
Oil lost in January. Apple lost in February. Tree lost in March. Snow lost in April. Apple lost in
Marcus born in January. Victoria born in February. George born in March. Brandon born in April. Jamie born in


1536

In [59]:
num_words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

anomolies = []
num_corr = 0
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    try:
        a = num_words.index(tok_logit_lens[9][0].replace(' ', ''))
        b= num_words.index(tok_logit_lens[10][0].replace(' ', ''))
        if int(a) < int(b):
            if tok_logit_lens[10][0] == pd['corr']:
                num_corr += 1
            else:
                anomolies.append(pd)
        else:
                anomolies.append(pd)
    except:
        anomolies.append(pd)
    # for i, tokouts in enumerate(tok_logit_lens):
    #     print(i-1, tokouts)
    # print('\n')

In [60]:
num_corr

1495

In [61]:
len(prompts_list)

1536

In [62]:
anomolies[0]

{'S1': 'May',
 'S2': 'June',
 'S3': 'July',
 'S4': 'August',
 'corr': ' September',
 'incorr': ' August',
 'text': 'Light done in May. Arm done in June. Road done in July. Book done in August. Ice done in'}

In [63]:
anomolies[-1]

{'S1': 'May',
 'S2': 'June',
 'S3': 'July',
 'S4': 'August',
 'corr': ' September',
 'incorr': ' August',
 'text': 'Aaron born in May. Chris born in June. Elizabeth born in July. April born in August. Gene born in'}

In [64]:
for pd in anomolies[:2]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' particular', ' vain']
2 [' order', ' the', ' conjunction', ' vain', ' accordance']
3 [' order', ' the', ' vain', ' accordance', ' conjunction']
4 [' order', ' the', ' conjunction', ' accordance', ' vain']
5 [' order', ' lieu', ' vain', ' late', ' conjunction']
6 [' order', ' lieu', ' June', ' conjunction', ' late']
7 [' June', ' September', ' May', ' March', ' July']
8 [' September', ' August', ' July', ' June', ' October']
9 [' September', ' October', ' November', ' August', ' Sept']
10 [' September', ' October', ' November', ' August', ' December']
11 [' September', ' October', ' November', ' August', ' December']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' particular', ' my']
2 [' order', ' the', ' conjunction

In [65]:
for pd in anomolies[-2:]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' particular', ' a']
2 [' the', ' order', ' a', ' accordance', ' front']
3 [' the', ' order', ' accordance', ' a', ' his']
4 [' front', ' the', ' order', ' conjunction', ' his']
5 [' order', ' lieu', ' late', ' 1968', ' front']
6 [' lieu', ' order', ' June', ' conjunction', ' front']
7 [' June', ' September', ' August', ' July', ' May']
8 [' September', ' August', ' July', ' October', ' November']
9 [' September', ' October', ' November', ' December', ' August']
10 [' September', ' October', ' November', ' December', ' August']
11 [' September', ' October', ' November', ' December', ' January']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' particular', ' a']
2 [' order', ' the', ' a', ' accordance', ' conjunction']
3

# dig 16

In [11]:
import pickle

task = "digits"
prompts_list = []

temps = ['done', 'lost', 'names']

for i in temps:
    file_name = f'/content/{task}_prompts_{i}.pkl'
    with open(file_name, 'rb') as file:
        filelist = pickle.load(file)

    print(filelist[0]['text'])
    prompts_list += filelist [:512] #768 512

len(prompts_list)

Van done in 1. Hat done in 2. Ring done in 3. Desk done in 4. Sun done in
Oil lost in 1. Apple lost in 2. Tree lost in 3. Snow lost in 4. Apple lost in
Marcus born in 1. Victoria born in 2. George born in 3. Brandon born in 4. Jamie born in


1536

In [12]:
num_corr = 0
anomolies = []
for pd in prompts_list:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    try:
        a = tok_logit_lens[9][0].replace(' ', '')
        b= tok_logit_lens[10][0].replace(' ', '')
        if int(a) < int(b):
            if tok_logit_lens[10][0] == pd['corr']:
                num_corr += 1
            else:
                anomolies.append(pd)
        else:
                anomolies.append(pd)
    except:
        anomolies.append(pd)
    # for i, tokouts in enumerate(tok_logit_lens):
    #     print(i-1, tokouts)
    # print('\n')
num_corr

1531

In [13]:
num_corr

1531

In [14]:
len(prompts_list)

1536

In [15]:
anomolies[0]

{'S1': '3',
 'S2': '4',
 'S3': '5',
 'S4': '6',
 'corr': ' 7',
 'incorr': ' 6',
 'text': 'Orange done in 3. Ocean done in 4. Sun done in 5. Bug done in 6. Queen done in'}

In [16]:
anomolies[-1]

{'S1': '3',
 'S2': '4',
 'S3': '5',
 'S4': '6',
 'corr': ' 7',
 'incorr': ' 6',
 'text': 'Lake lost in 3. Ear lost in 4. Key lost in 5. Hat lost in 6. Fire lost in'}

In [17]:
for pd in anomolies[:2]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' conjunction', ' a']
1 [' order', ' the', ' conjunction', ' particular', ' a']
2 [' order', ' the', ' vain', ' front', ' conjunction']
3 [' order', ' the', ' vain', ' conjunction', ' front']
4 [' order', ' the', ' conjunction', ' vain', ' spite']
5 [' order', ' conjunction', ' 3', ' vain', ' 15']
6 [' 3', ' 5', ' 2', ' 4', ' 7']
7 [' 3', ' 5', ' 7', ' 4', ' 2']
8 [' 7', ' 6', ' 5', ' 8', ' 1']
9 [' 7', ' 8', ' 6', ' 9', '7']
10 [' 7', ' 8', ' 9', ' 6', ' 1']
11 [' 7', ' 8', ' 1', ' 9', ' 6']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' conjunction']
1 [' order', ' the', ' conjunction', ' a', ' particular']
2 [' order', ' the', ' conjunction', ' vain', ' front']
3 [' order', ' the', ' conjunction', ' vain', ' accordance']
4 [' order', ' conjunction', ' the', ' relation', ' accordance']
5 [' order', ' 3', ' 5', ' 18', ' 6']
6 

In [18]:
for pd in anomolies[-2:]:
    test_text = pd['text']
    layer_logits = get_logits(test_text)
    tok_logit_lens = get_decoded_indiv_toks(layer_logits)
    for i, tokouts in enumerate(tok_logit_lens):
        print(i-1, tokouts)
    print('\n')

-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' order', ' the', ' particular', ' a', ' front']
1 [' order', ' the', ' a', ' conjunction', ' front']
2 [' order', ' the', ' a', ' vain', ' accordance']
3 [' order', ' the', ' vain', ' his', ' a']
4 [' order', ' the', ' spite', ' relation', ' vain']
5 [' order', ' 3', ' 2', ' 5', ' 1']
6 [' 3', ' 2', ' 5', ' 1', ' 4']
7 [' 3', ' 2', ' 1', ' 5', ' 4']
8 [' 5', ' 4', ' 3', ' 1', ' 2']
9 [' 5', ' 6', ' 1', ' 7', '5']
10 [' 5', ' 6', ' 1', ' 7', ' 3']
11 [' 5', ' 6', ' 1', ' 7', ' 3']


-1 [' challeng', ' mathemat', ' arrang', ' corrid', ' destro']
0 [' the', ' order', ' a', ' particular', ' front']
1 [' order', ' the', ' a', ' conjunction', ' particular']
2 [' order', ' the', ' a', ' accordance', ' vain']
3 [' order', ' the', ' accordance', ' relation', ' vain']
4 [' order', ' the', ' accordance', ' spite', ' relation']
5 [' order', ' 3', ' 5', ' 18', ' 15']
6 [' 3', ' 5', ' 2', ' 7', ' 4']
7 [' 3', ' 5', ' 7', ' 4', ' 6']
